In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv("/kaggle/input/esigning-of-loan-based-on-financial-history/financial_data.csv")

In [ ]:
df.head()

In [ ]:
df.columns

In [ ]:
df.describe()

In [ ]:
df.nunique()

In [ ]:
df.isna().sum()

In [ ]:
df.info()

In [ ]:
df.pay_schedule.value_counts()

In [ ]:
df = pd.get_dummies(df, "pay_schedule")

In [ ]:
df.drop(columns = ["e_signed"], axis = 1).corrwith(df.e_signed).plot.bar(figsize=(17,6), 
                                                 title="Correlation with Response Variable (e_signed)",
                                                 fontsize=15,
                                                 rot=45,
                                                 )
plt.grid(b=True, which='major', color='#666666', linestyle='-', alpha=.3)
plt.xticks(rotation=90)
plt.show()

In [ ]:
plt.figure(figsize=(20,20))
sns.heatmap(df.corr(), annot = True, fmt=".2f", linewidths=0.1, cbar=False)
plt.show()

In [ ]:
corr_matrix = df.corr()
filtre = np.abs(corr_matrix["e_signed"]>0.03)
corr_columns = corr_matrix.columns[filtre].tolist()
corr = df[corr_columns].corr()

plt.figure(figsize=(17,12))
sns.heatmap(corr, annot = True, linewidths=.4, linecolor="w",fmt=".2f")
plt.show()

In [ ]:
np.abs(df.corr()["e_signed"]).sort_values(ascending=False)

In [ ]:
df.columns

In [ ]:
df.drop(columns = ['pay_schedule_weekly', 'pay_schedule_monthly'], axis = 1, inplace = True)

In [ ]:
df.iloc[:, 0:-2].hist(bins = 20, figsize = (20,30), color = "g",)
plt.show()

In [ ]:
# Distribution plots for our continuous variables
# unpivot DataFrame into long-form
con_1 = pd.melt(df, id_vars=None, value_vars=None, var_name=None, value_name='value', col_level=None, ignore_index=True)

# Seaborn customizations
sns.set_theme(style="darkgrid", font='serif', context='notebook')

# initializing our FacetGrid object and using map to create subplots
g = sns.FacetGrid(con_1, col='variable', col_wrap=3, sharex=False, sharey=False, height=4)
g = g.map(sns.distplot, 'value', color='r')

# Matplotlib customization
g.set_xticklabels(rotation=45)
plt.show()

In [ ]:
# Distribution plots for our continuous variables
# unpivot DataFrame into long-form
con_1 = pd.melt(df, id_vars=None, value_vars=None, var_name=None, value_name='value', col_level=None, ignore_index=True)

# Seaborn customizations
sns.set_theme(style="darkgrid", font='serif', context='notebook')

# initializing our FacetGrid object and using map to create subplots
g = sns.FacetGrid(con_1, col='variable', col_wrap=3, sharex=False, sharey=False, height=4)
g = g.map(sns.kdeplot, 'value', color='b')

# Matplotlib customization
g.set_xticklabels(rotation=45)
plt.show()

In [ ]:
df["personal_account_months"] = df["personal_account_m"] + df["personal_account_y"]*12

In [ ]:
df[["personal_account_y", "personal_account_m", "personal_account_months"]].head()

In [ ]:
df.drop(columns = ["personal_account_m", "personal_account_months", "months_employed"], axis = 1, inplace = True)

In [ ]:
response = df["e_signed"]
users = df["entry_id"]

In [ ]:
df.drop(columns=["entry_id", "e_signed"], axis = 1, inplace = True)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df,
                                                   response,
                                                   test_size=0.33,
                                                   random_state=0)

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [ ]:
sc_X = StandardScaler()
X_train_sc = pd.DataFrame(sc_X.fit_transform(X_train), 
                          columns = X_train.columns.values, 
                         index = X_train.index.values)

X_test_sc = pd.DataFrame(sc_X.transform(X_test), 
                         columns = X_test.columns.values, 
                        index = X_test.index.values)

In [ ]:
X_train_sc.head()

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, recall_score, precision_score 

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
lr = LogisticRegression(penalty='l2', random_state=0)
lr.fit(X_train_sc, y_train)

In [ ]:
y_pred_lr = lr.predict(X_test_sc)

In [ ]:
print(accuracy_score(y_test, y_pred_lr))
print()
print(precision_score(y_test, y_pred_lr))
print()
print(recall_score(y_test, y_pred_lr))
print()
print(f1_score(y_test, y_pred_lr))
print()
print(confusion_matrix(y_test, y_pred_lr))

In [ ]:
from sklearn.svm import SVC

In [ ]:
svm = SVC(random_state=0, kernel="rbf")
svm.fit(X_train_sc, y_train)

In [ ]:
y_pred_svm = svm.predict(X_test_sc)

In [ ]:
print(accuracy_score(y_test, y_pred_svm))
print()
print(precision_score(y_test, y_pred_svm))
print()
print(recall_score(y_test, y_pred_svm))
print()
print(f1_score(y_test, y_pred_svm))
print()
print(confusion_matrix(y_test, y_pred_svm))

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rf = RandomForestClassifier(random_state = 0,
                           n_estimators = 100,
                           criterion = "entropy")
rf.fit(X_train_sc, y_train)

In [ ]:
y_pred_rf = rf.predict(X_test_sc)

In [ ]:
print(accuracy_score(y_test, y_pred_rf))
print()
print(precision_score(y_test, y_pred_rf))
print()
print(recall_score(y_test, y_pred_rf))
print()
print(f1_score(y_test, y_pred_rf))
print()
print(confusion_matrix(y_test, y_pred_rf))

In [ ]:
from sklearn.model_selection import cross_val_score

In [ ]:
cv_result = []
best_estimator = []
estimator = [lr, svm, rf]

for i in range(len(estimator)):
    accuracies = cross_val_score(estimator = estimator[i], X=X_train_sc, y=y_train, cv=5, n_jobs=-1, verbose=True)
    cv_result.append(accuracies.mean())
    best_estimator.append(estimator[i])

In [ ]:
result = pd.DataFrame({"Cross Validation Score" : cv_result, "ML Models" : ["LogisticRegression","SVM","RandomForestClassifier"]})
plt.figure(figsize=(16,7))
sns.barplot(y = "Cross Validation Score", x ="ML Models", data= result)
plt.show()

In [ ]:
result

In [ ]:
parameters={"max_depth": [2, 3, None],
           "max_features" : [1, 3, 5, 7, 10],
           "min_samples_split" : [2, 3, 5, 10, 11],
           "min_samples_leaf" : [1, 3, 5, 10],
           "bootstrap" : [True, False],
           "criterion" : ["entropy"]}

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
grid_search = GridSearchCV(estimator = rf,
                          param_grid = parameters,
                          scoring = "accuracy",
                          cv = 3,
                          n_jobs = -1,
                          verbose = 2)

In [ ]:
grid_search.fit(X_train_sc, y_train)

In [ ]:
rf_best_accuracy = grid_search.best_score_
rf_best_parameters = grid_search.best_params_

In [ ]:
print(rf_best_accuracy)

In [ ]:
print(rf_best_parameters)

In [ ]:
parameters_2={"max_depth": [1, None],
           "max_features" : [9, 10, 11],
           "min_samples_split" : [2],
           "min_samples_leaf" : [9, 10, 11],
           "bootstrap" : [True],
           "criterion" : ["entropy",  "gini"]}

In [ ]:
grid_search = GridSearchCV(estimator = rf,
                          param_grid = parameters_2,
                          scoring = "accuracy",
                          cv = 3,
                          n_jobs = -1,
                          verbose = 1)